# Immigration Data Warehouse
### Data Engineering Project

#### Project Summary
This project aims to be able to answers questions around US immigration. We extract data from three different data sources, the I94 immigration dataset of 2016, city temperature data from Kaggle and US city demographic data from OpenSoft. We have designed 3 dimension tables: dimTemperature, dimImmigration , dimDemographics and one fact table: factImmigration. We use Spark for ETL jobs and store the results in parquet for downstream analysis.



In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, year, month, col, isnull,avg,monotonically_increasing_id, isnan, when, count
import numpy as np
from pyspark.sql.types import DateType
from datetime import datetime, timedelta
from glob import glob
from pyspark.sql.types import StringType, IntegerType

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"

In [ ]:
spark = SparkSession.builder.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
The goal of this project is ingest data from three different data sources and create fact and dimension table to be able to do analysis on US immigration using factors of city, average temperature, city demographics and seasonality.

#### Describe and Gather Data 

**I94 Immigration Data**: comes from the U.S. National Tourism and Trade Office and contains various statistics on international visitor arrival in USA and comes from the US National Tourism and Trade Office. The dataset contains data from 2016.<br>
**World Temperature Data**: comes from Kaggle and contains average weather temperatures by city. <br>
**U.S.City Demographic Data**: comes from OpenSoft and contains information about the demographics of all US cities such as average age, male and female population. 


In [ ]:
# Read in the data here
# Read temperature data
temparatureData = spark.read.format("csv").option("header", "true").load("../../data2/GlobalLandTemperaturesByCity.csv")
# Read immigeration data
immigrationData = spark.read.load('./sas_data')
# Read demographics data
demographyData = spark.read.format("csv").option("delimiter", ";").option("header", "true").load( "us-cities-demographics.csv")

In [ ]:
immigrationData.limit(5).toPandas()

In [ ]:
temparatureData.limit(5).toPandas()

In [ ]:
demographyData.limit(5).toPandas()

### Step 2: Explore and Assess the Data


In [ ]:
# Create udf to convert SAS date to PySpark date 
@udf(StringType())
def convert_datetime(x):
    if x:
        return (datetime(1960, 1, 1).date() + timedelta(x)).isoformat()
    return None


In [ ]:
#Immigration Data Cleaning and Staging

#creating alias name 
immigrationDatadf = immigrationData.filter(immigrationData.i94addr.isNotNull()).filter(immigrationData.cicid.isNotNull()).withColumn("Year",col("i94yr").cast("integer")).withColumn("Month",col("i94mon").cast("integer"))
cleaned_immigrationData = immigrationDatadf.withColumn("arrdate", convert_datetime(immigrationDatadf.arrdate))
staging_immigrationData = cleaned_immigrationData.select(col("cicid").alias("id"), 
                                       col("arrdate").alias("date"),
                                       col("i94port").alias("city_code"),
                                       col("i94addr").alias("state_code"),
                                       col("i94bir").alias("age"),
                                       col("Year").alias("year_of_arrival"),
                                       col("Month").alias("month_of_arrival"),
                                       col("gender").alias("gender"),
                                       col("visatype").alias("visa_type"),
                                       "count").drop_duplicates()

staging_immigrationData.show()
staging_immigrationData.createOrReplaceTempView("stgImmigration")


In [ ]:
#Immigration Data Cleaning
temparatureData_Cleansed = temparatureData.filter(temparatureData.Country=='United States').filter(temparatureData.AverageTemperatureUncertainty.isNotNull()).filter(temparatureData.AverageTemperature.isNotNull())
temparatureData_Cleansed=temparatureData_Cleansed.withColumn("AvgTemp",col("AverageTemperature").cast("float")).withColumn("AvgDifferenceinTemp",col("AverageTemperatureUncertainty").cast("float"))


In [ ]:
# Demography data Staging
stage_demographyData = demographyData.withColumn("median_age", demographyData['Median Age']) \
    .withColumn("pcnt_male_pop", (demographyData['Male Population'] / demographyData['Total Population']) * 100) \
    .withColumn("pcnt_female_pop", (demographyData['Female Population'] / demographyData['Total Population']) * 100) \
    .withColumn("pcnt_foreign_born", (demographyData['Foreign-born'] / demographyData['Total Population']) * 100).withColumn("state_code", (demographyData['State Code'])).withColumn("total_pop", (demographyData['Total Population'])) 


stage_demographyData.show()


In [ ]:
# Creating Dimension Table
staging_immigrationData.createOrReplaceTempView("dimImmigration")
dimImmigration = spark.sql('''SELECT id,state_code,city_code,visa_type,year_of_arrival,month_of_arrival FROM dimImmigration''')
dimImmigration.show()

In [ ]:
# Creating Dimension Table
dimTemperature=temparatureData_Cleansed.groupBy("Country","City").agg({'AvgTemp':'avg', 'AvgDifferenceinTemp':'avg'})
dimTemperature=dimTemperature.select("Country","City",col("avg(AvgTemp)").alias("AvgTemp"),col("avg(AvgDifferenceinTemp)").alias("AvgDifferenceinTemp"))
dimTemperature.createOrReplaceTempView("dimTemperature")
dimTemperature.show()

In [ ]:
# Creating Dimension Table
stage_demographyData.createOrReplaceTempView("dimDemography")
dimDemography = spark.sql('''SELECT state_code,state,city,median_age,pcnt_male_pop,pcnt_female_pop,pcnt_foreign_born,total_pop FROM dimDemography''')
dimDemography.show()


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

**DIMENSION TABLES**

1.**dimImmigration** : It contains immigration events

* id               - Unique Identifier
* state_code       - State Code
* city_code        - City Code 
* visa_type        - Type of visa issues
* year_of_arrival  - Year of Arrival to US
* month_of_arrival - Month of Arrival to US

2.**dimTemperature** :  It contains average temparature city in US

* Country             - Country Name
* City                - City Name
* AvgTemp             - Average Temperature in City
* AvgDifferenceinTemp - Average Variation In Temperature

3.**dimDemography** : It has information on Demographic Statistics

* state_code         - State Code
* city               - City Name
* median_age         - Median age of people in city
* pcnt_male_pop      - Percentage of Male Population
* pcnt_female_pop    - Percentage of Female Population
* pcnt_foreign_born  - Percentage of People who are born outside US 
* total_pop          - Total Polulation

4.**factImmigration** : The Fact table gives the count of entry into US soil.

* factImmigration_pk  - Unique Identifier
* id                  - Unique Identifier of dimImmigration
* city                - City Name
* state_code          - State Code




#### 3.2 Mapping Out Data Pipelines
1.dimTemperature is created by reading data from csv file and aggregated data after cleaning (Refer Step 2)

2.dimImmigration is created from list of sas files and cleaned (Refer Step 2)

3.dimDemography is created by reading data from csv file after cleaning tables (Refer Step 2)

4.factImmigration is created by joining staging_immigrationData,dimDemography and dimTemperature tables

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# factImmigration is created by joining staging_immigrationData,dimDemography and dimTemperature tables
factImmigration = spark.sql('''
SELECT stgImmigration.id,
       dimTemperature.City,
       dimDemography.state_code,
       stgImmigration.count
FROM dimTemperature  
JOIN dimDemography  ON (dimTemperature.city = dimDemography.city)
JOIN stgImmigration ON ( dimDemography.state_code=stgImmigration.state_code)

''')

In [ ]:
# Adding Sequencial Primary Key to Fact Table
factImmigration=factImmigration.withColumn("factImmigration_pk", monotonically_increasing_id()) 
factImmigration.show()

In [ ]:
# Write Dimention Tables as Parquet File
dimTemperature.write.mode("append").partitionBy("City").parquet("/target/dimTemperature.parquet")

In [ ]:
# Write Dimention Tables as Parquet File
dimDemography.write.mode("append").partitionBy("city").parquet("/target/dimDemography.parquet")

In [ ]:
# Write Dimention Tables as Parquet File
dimImmigration.write.mode("append").partitionBy("city_code").parquet("/target/dimImmigration.parquet")

In [ ]:
# Write Dimention Tables as Parquet File
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")
factImmigration.write.mode("append").partitionBy("City").parquet("/target/factImmigration.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
#check dimTemperature counts
dimTemperature.count()

In [ ]:
#check dimImmigration counts
dimImmigration.count()

In [ ]:
#check dimDemography counts
dimDemography.count()

In [ ]:
#check factImmigration counts
factImmigration.count()

In [ ]:
#check dimTemperature primary key has null
dimTemperature.select([count(when(isnan('City'),True))]).show()

In [ ]:
#check dimImmigration primary key has null
dimImmigration.select([count(when(isnan('id'),True))]).show()

In [ ]:
#check dimDemography primary key has null
dimDemography.select([count(when(isnan('state_code'),True))]).show()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. 

**DIMENSION TABLES**

1.**dimImmigration** : It contains immigration events

* id               - Unique Identifier
* state_code       - State Code
* city_code        - City Code 
* visa_type        - Type of visa issues
* year_of_arrival  - Year of Arrival to US
* month_of_arrival - Month of Arrival to US

2.**dimTemperature** :  It contains average temparature city in US

* Country             - Country Name
* City                - City Name
* AvgTemp             - Average Temperature in City
* AvgDifferenceinTemp - Average Variation In Temperature

3.**dimDemography** : It has information on Demographic Statistics

* state_code         - State Code
* city               - City Name
* median_age         - Median age of people in city
* pcnt_male_pop      - Percentage of Male Population
* pcnt_female_pop    - Percentage of Female Population
* pcnt_foreign_born  - Percentage of People who are born outside US 
* total_pop          - Total Polulation

4.**factImmigration** : The Fact table gives the count of entry into US soil.

* factImmigration_pk  - Unique Identifier
* id                  - Unique Identifier of dimImmigration
* city                - City Name
* state_code          - State Code


#### Step 5: Complete Project Write Up
* **Clearly state the rationale for the choice of tools and technologies for the project.**
PySpark is chosen for this project as it is known for processing large amount of data fast (with in-memory compute), scale easily with additional worker nodes, with ability to digest different data formats (e.g. SAS, Parquet, CSV), and integrate nicely with cloud storage like S3 and warehouse like Redshift.Python provides an additional edge to spark native scala with its enormous library size.
* **Propose how often the data should be updated and why.**
It Depends on the Data Availability at source and Reporting Cycle. If the data at source is available bi-monthly we can perform a complete refresh once a fortnight.
* **Write a description of how you would approach the problem differently under the following scenarios:**
 * **The data was increased by 100x.**
We can consider spinning up larger instances of EC2s hosting Spark and/or additional Spark work nodes. With added capacity arising from either vertical scaling or horizontal scaling, we should be able to accelerate processing time.
 * **The data populates a dashboard that must be updated on a daily basis by 7am every day.**
We can consider using Airflow/Oozie to schedule and automate the data pipeline jobs.
 * **The database needed to be accessed by 100+ people.**
We can use AWS Redshift With Concurrency Scaling feature, which can support virtually unlimited concurrent users and concurrent queries, with consistently fast query performance. When concurrency scaling is enabled, Amazon Redshift automatically adds additional cluster capacity when you need it to process an increase in concurrent read queries. Write operations continue as normal on your main cluster. Users always see the most current data, whether the queries run on the main cluster or on a concurrency scaling cluster.